In [3]:
pip install rapidfuzz

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 1.3 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 1.5 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.5 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import re
def clean_and_tokenize(text):
    # Remove punctuation including Devanagari danda, keep only letters and digits
    cleaned = re.sub(r'[^\u0900-\u0963\u0966-\u097F\d\s]+', '', text)
    cleaned = re.sub(r'[।॥]+', '', cleaned)  # Explicitly remove danda
    return [token for token in cleaned.split() if token]

def normalize(text):
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)  # Keep only Devanagari + spaces
    text = re.sub(r'[।॥]+', '', text)  # Explicitly remove danda
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [1]:
# Read files
with open('real.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

with open('infer.txt', 'r', encoding='utf-8') as f:
    asr_output = [line.strip() for line in f if line.strip()]

In [34]:
from rapidfuzz import fuzz, process

# ground_truth = normalize(full_text)
ground_truth = full_text
asr_chunks = [normalize(chunk) for chunk in asr_output]

def find_best_match(asr_text, ground_truth, threshold=70):
    # Split ground truth into candidate windows
    words = ground_truth.split()
    candidates = []
    
    asr_len = len(asr_text.split())
    
    # Generate sliding windows of similar length
    for i in range(len(words)):
        for window_size in range(max(1, asr_len - 5), asr_len + 10):
            if i + window_size <= len(words):
                candidate = ' '.join(words[i:i+window_size])
                candidates.append((candidate, i, i+window_size))
    
    # Find best match
    best_match = None
    best_score = 0
    
    for candidate, start_idx, end_idx in candidates:
        score = fuzz.ratio(asr_text, candidate)
        if score > best_score:
            best_score = score
            best_match = (candidate, start_idx, end_idx, score)
    
    if best_score >= threshold:
        return best_match
    return None


print("confidence  text_match\n------------------------")
texts = []
for chunk in asr_chunks:
    match = find_best_match(chunk,ground_truth)
    print(f"{match[3]:.2f}: {match[0]}")
    texts.append(match[0])
    

with open("matched1.txt", "w", encoding="utf-8") as f:
    for line in texts:
        f.write(line + "\n")

confidence  text_match
------------------------
100.00: काठमाडौँ
80.00: जब–जब म गाउँ पुग्छु,
90.45: सुन्दर स्मृतिहरू फिल्मका दृश्यझैं चल्मलाउन थाल्छन्। सुसाइरहेका आँधीखोलामा बाल्यकालीन स्वरहरू सुनिएझैं लाग्न थाल्छन्। सहर पसेसँगै छुटेका बाल्यकाल, विकास भित्रिएसँगै हराएका गोरेटो
85.71: र स्याङ्जा पुतलीबजारको ऊबेलाका अर्ग्यानिक सौन्दर्य— यी सबै–सबै सजीव भएर आउँछन्। यो तिहारमा १७ वर्षपछि गाउँमा भाइटीका लगाइयो। दिदीबहिनी, आफन्तहरू र साथीभाइसँगको जमघटले न्यानो महसुस गराएसँगै सम्झन
89.47: सम्झन थालें ती दिन।
94.02: जुन दिनहरू यो उमेरको लेन्सबाट चिहाउँदा ‘स्वर्ग’ जस्ता लाग्थे।
93.58: बिदा भन्ने शब्द सुन्नासाथ हाम्रो कान निकै चनाखो हुन्थे।
72.73: जब कि
92.50: पूरा कक्षाभरि सरले पढाएका कुनै पनि अक्षर
89.47: एउटा कानले सुनेर अर्कोले उडाइन्थ्यो। यही कारण एक पटक सरको रामधुलाइ भेट्यौं । कहिलेकाहीं पढाइ भइरहेको समयमा स्कुलमा हाफ छुट्टीको सूचना टुप्लुक्क आइपुग्थ्यो
89.29: कक्षा ८ मा पढ्दाको कुरा, त्यतिबेला मलगायत केही साथी कक्षाको अन्तिम बेन्चतिर बस्थ्यौं।
91.85: अवधेश झा सर तेस्रो घण्टीको कक्षामा विज

In [18]:
def extract_original_span(original_text, start_word, end_word):
    words = original_text.split()
    return ' '.join(words[start_word:end_word])

extract_original_span(full_text,341,355)

'। मेरो जिद्दीअघि बा–आमाको के लाग्नू ? दसैं–तिहारको लामो बिदामा हामी गाउँको युवा पुस्तकालय'

In [ ]:
def find_best_match(asr_text, ground_truth, threshold=70):
    words = ground_truth.split()
    asr_len = len(asr_text.split())
    
    best_match = None
    best_score = 0
    
    for i in range(len(words)):
        for window_size in range(max(1, asr_len - 5), asr_len + 10):
            if i + window_size <= len(words):
                candidate = ' '.join(words[i:i+window_size])
                score = fuzz.ratio(asr_text, candidate)
                if score > best_score:
                    best_score = score
                    best_match = (i, i+window_size, score)
    
    if best_score >= threshold:
        return best_match
    return None

def extract_original_span(original_text, start_word, end_word):
    words = original_text.split()
    return ' '.join(words[start_word:end_word])

# Read files
with open('real.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

with open('infer.txt', 'r', encoding='utf-8') as f:
    asr_output = [line.strip() for line in f if line.strip()]

# Process
ground_normalized = normalize(full_text)
results = []

for idx, asr_line in enumerate(asr_output):
    asr_norm = normalize(asr_line)
    match = find_best_match(asr_norm, ground_normalized, threshold=70)
    
    if match:
        start_idx, end_idx, score = match
        original_span = extract_original_span(full_text, start_idx, end_idx)
        results.append({
            'chunk_id': idx,
            'asr_text': asr_line,
            'matched_text': original_span,
            'score': score
        })
        print(f"[{idx}] Score: {score:.1f}")
    else:
        print(f"[{idx}] NO MATCH")
        results.append({
            'chunk_id': idx,
            'asr_text': asr_line,
            'matched_text': None,
            'score': 0
        })

[0] Score: 100.0
[1] Score: 84.2
[2] Score: 91.2
[3] Score: 86.9
[4] Score: 91.9
[5] Score: 96.5
[6] Score: 94.4
[7] Score: 72.7
[8] Score: 92.5
[9] Score: 90.4
[10] Score: 90.4
[11] Score: 93.5
[12] Score: 97.2
[13] Score: 85.2
[14] Score: 82.8
[15] Score: 91.7
[16] Score: 95.5
[17] Score: 88.8
[18] Score: 90.1
[19] Score: 91.2
[20] Score: 91.1
[21] Score: 84.8
[22] Score: 87.4
[23] Score: 89.9
[24] Score: 87.5
[25] Score: 79.2
[26] Score: 94.3
[27] Score: 96.0
[28] Score: 92.3
[29] Score: 89.3
[30] Score: 92.5
[31] Score: 89.8
[32] Score: 94.0
[33] Score: 83.8
[34] Score: 90.9
[35] Score: 80.3
[36] Score: 91.4
[37] Score: 96.6
[38] Score: 97.9


In [20]:
results

[{'chunk_id': 0,
  'asr_text': 'काठमाडौँ',
  'matched_text': 'काठमाडौँ',
  'score': 100.0},
 {'chunk_id': 1,
  'asr_text': 'जबजाबामा गाउँ पुग्छो',
  'matched_text': '— जब–जब म गाउँ',
  'score': 84.21052631578947},
 {'chunk_id': 2,
  'asr_text': 'सुन्दर स्मृतिहरू फिल्मका दी सेल छैँ चल्मलाउन थाल्छन् सुसाइरहेका आँधी खोलामा बाल्यकालीन स्वरहरू सुनिय छहैँ लाग्न थाल्छन् सहरपसेसँगै छुटेका बाल्यकाल विकास भित्रीय सँगै हराइका गोरेटो',
  'matched_text': 'पुग्छु, सुन्दर स्मृतिहरू फिल्मका दृश्यझैं चल्मलाउन थाल्छन्। सुसाइरहेका आँधीखोलामा बाल्यकालीन स्वरहरू सुनिएझैं लाग्न थाल्छन्। सहर पसेसँगै छुटेका बाल्यकाल, विकास भित्रिएसँगै हराएका',
  'score': 91.21813031161473},
 {'chunk_id': 3,
  'asr_text': 'र स्याङ्जा पोतली बजारको उबेलाक अर्ज्ञानिक यी सबै सबै सजि भएर आउँछन्। यो तिहारमा सत्र वर्ष पछि गाउँमा भाइटीका लगाइ यो दिदीबहिनी आफन्तहरू र साथी भाइ सङ्घको जम् घटले न्यहानो महसोस गराएसँगै',
  'matched_text': 'गोरेटो र स्याङ्जा पुतलीबजारको ऊबेलाका अर्ग्यानिक सौन्दर्य— यी सबै–सबै सजीव भएर आउँछन्। यो तिहारमा १७ व